Copyright 2023 Province of British Columbia

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at 

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under he License.
the License.

## Imports and Parameters

In [1]:
# add our stuff to the path
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# other stuff
import time

# import our stuff
from importlib import reload
from src import connect, preprocess, synthetic, model

In [2]:
# PARAMETERS

# info to gain access to database, IDIR restricted 
CRED_PATH = '../credentials.txt'

# where model is stored. requires credentials.txt for full path 
MODEL_BASE_PATH = 'Model/Q32'

# which tables to access
RESPONSE_TABLE = 'dbo.AQ32RACE'
CODE_TABLE = 'dbo.AQ32RACE_Codes'
CLOSED_TABLE = 'dbo.Q32RACEMultiResponse'
RESULTS_TABLE = 'AQ32RACE_RESULTS'

# which column to use/create 
RESPONSE_COLUMN = 'aq32race_combined'
OUTPUT_COLUMNS = 'q32race_c'
N_COLUMNS = 16

# delimiter to send back with concatenated results
DELIMITER = 'μ' 

# amount of synthetic data to produce
N_SYNTHETIC_MIXED = 50_000

# threshold for accepting as a flagged category
THRESHOLD = 0.5

# upper and lower thresholds for flagging as a possible category
TENTATIVE_UPPER = 0.75
TENTATIVE_LOWER = 0.25

## Read from Database

In [3]:
# Read in all data required to build model
connection = connect.create_connection(CRED_PATH)

# actual responses
df_open = connect.fetch_table(RESPONSE_TABLE, connection)

# codes to match
code_df = connect.fetch_table(CODE_TABLE, connection)

# closed respones (for multi response frequencies)
df_closed = connect.fetch_table(CLOSED_TABLE, connection)

C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\connect.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', connection)


## Preprocess Data

* Use only cycle 1 for training.
* Use the cleaned column.
* Correct spelling where possible (takes longer time to run).
* Create input and output tables for model (input table takes a while to produce)

In [4]:
# for training purposes, open responses should only be those from cycle 1
df = df_open[df_open.cycle == 1].reset_index(drop=True)
df.head()

,id,q32race,aq32race,aq32race_cleaned,coding_comment,q32race_c01,q32race_c02,q32race_c03,q32race_c04,q32race_c05,...,q32race_c08,q32race_c09,q32race_c10,q32race_c11,q32race_c12,q32race_c13,q32race_c14,q32race_c15,q32race_c16,cycle
0,0038702,97,Canadian,None,None,97,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1
1,0000004,97,Canadian,None,None,20001,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1
2,0000013,97,White,None,None,105,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1
3,0000047,97,Caucasian,None,None,105,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1
4,0000073,108µ97,Brazilian,None,None,108,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1


In [ ]:
# get a cleaned up column to use (mix of actual comment column and cleaned)
# question : will the cleaned column always be available? or add in extra preprocessing here
df.loc[:, RESPONSE_COLUMN] = df.apply(
    lambda x: x.aq32race.lower() if x.aq32race_cleaned == None or x.aq32race_cleaned=='105' else x.aq32race_cleaned.lower(), 
    axis=1
)

# fix the spelling to send to the model 
# note this takes a long time to do, so don't add it to the get_scores function
df.loc[:, RESPONSE_COLUMN] = df[RESPONSE_COLUMN].apply(preprocess.correct_spelling)

df.head()

In [ ]:
# get long form table of codes
code_df_long = preprocess.get_long_form_codes(code_df)
code_df_long.head()

In [ ]:
# build a training dataset for the model 
# first part of dataset: actual data

# INPUTS TO MODEL
headers = list(preprocess.get_scores('test', code_df_long, as_df = True).col_id.values)

input_df = preprocess.get_scores_from_df(df, RESPONSE_COLUMN, code_df_long, headers=headers)
display(input_df.head())

# OUTPUTS OF MODEL
# converts the coded columns into wide form 1/0 binary responses for every option 
output_df = preprocess.get_outputs_wide(df, RESPONSE_COLUMN, code_df_long, OUTPUT_COLUMNS, N_COLUMNS)
output_df.head()

## Create Synthetic Data

* To augment our training data, produce synthetic data from the available phrases in the code list. 
* Both singular phrases and multi-response phrases are produced.
* The multi-response phrases are randomly generated according to weights associated with the non-written responses. 

In [ ]:
# create synthetic data
# this section will create synthetic data that matches a single category based on available phrases 
input_columns = list(input_df.columns)
output_columns = list(output_df.columns)

extra_input_df, extra_output_df = synthetic.create_single_phrase_synthetic(
    output_df, 
    input_columns,
    output_columns,
    code_df_long
)

In [ ]:
# create synthetic data
# this section will create synthetic data that matches multiple categories
mixed_input_df, mixed_output_df = synthetic.create_multi_phrase_synthetic(
    output_df,
    df_closed,
    input_columns,
    output_columns,
    code_df_long,
    N_SYNTHETIC_MIXED
)

In [ ]:
# Concatenate with existing data
final_input_df = pd.concat([input_df, extra_input_df, mixed_input_df], ignore_index=True).drop('response', axis=1).astype(int)
final_output_df = pd.concat([output_df, extra_output_df, mixed_output_df], ignore_index=True).drop('response', axis=1).astype(int)

## Train Model

To deal with the fact we have many possible categorical outputs, use a simple random forest model that will handle multiple outputs better than other models.

* After training, save model. 

In [ ]:
clf = model.create_model(final_input_df, final_output_df)

In [ ]:
model.save_model(CRED_PATH, MODEL_BASE_PATH, clf, code_df_long)

In [ ]:
clf, code_df_long = model.load_model(CRED_PATH, MODEL_BASE_PATH)

## Produce Model Results

* Look at some results by hand to see if it makes sense
* Create results for entire hand-coded dataset
* Send results back to databse

In [ ]:
sentence = 'chinese, japanese, korean'
model.list_classes(sentence, code_df_long, clf, spellcheck=True)

In [ ]:
results_df = model.produce_results(
    df, input_df, output_df, 
    clf,
    OUTPUT_COLUMNS,
    N_COLUMNS,
    threshold=THRESHOLD,
    tentative_lower = TENTATIVE_LOWER,
    tentative_upper = TENTATIVE_UPPER,
    delimiter = DELIMITER
)

In [ ]:
# save back to database
engine = connect.create_connection(CRED_PATH, sqlalchemy=True)

# for initial save of cycle 1, always replace. any subsequent inputs should be appended
connect.save_table(results_df, RESULTS_TABLE, engine, how='replace')